# Customized scraper for Australian indeed 

### Importing packages 

In [ ]:
from selenium import webdriver
import pandas as pd 
from bs4 import BeautifulSoup
import requests
from selenium.common.exceptions import ElementClickInterceptedException, TimeoutException, NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait

from lxml.html import fromstring
import requests
import urllib 
from itertools import cycle
import traceback

### Scraper

In [ ]:
#download chrome driver and put it in the same folder as script, and give it you path :-) 
driver = webdriver.Chrome('/Users/carolinetvergaard/Desktop/NLP/scraping/chromedriver')

#define dataframe columns 
dataframe = pd.DataFrame(columns=["Title","Location","Company","Salary","Description", "Occupation", "Sector"])

for i in range(0,600,2): # this goes from 0 to n with increments of i (tried to change to avoid duplicates)
    
    driver.get("https://au.indeed.com/jobs?q=media&start="+str(i))
    
    driver.implicitly_wait(7)

    all_jobs = driver.find_elements_by_class_name('result')
    
    for job in all_jobs:
        result_html = job.get_attribute('innerHTML')
        soup = BeautifulSoup(result_html,'html.parser')
    try:
        title = soup.find("a",class_="jobtitle").text.replace('\n','')
    except:
        title = 'None'

    try:
        location = soup.find(class_="location").text
    except:
        location = 'None'

    try:
        company = soup.find(class_="company").text.replace("\n","").strip()
    except:
        company = 'None'

    try:
        salary = soup.find(class_="salary").text.replace("\n","").strip()
    except:
        salary = 'None'
    
    sum_div = job.find_elements_by_class_name("summary")[0]
    
    try:
        sum_div.click()
    except:
        close_button = driver.find_elements_by_class_name("popover-x-button-close")[0]
        close_button.click()
        sum_div.click()
        
    try:
        job_desc = driver.find_element_by_id('vjs-desc').text
    except:
        job_desc = 'None'
    occupation = "Media Relations" #edit
    sector = "Creative Arts & Design" #edit 

    dataframe = dataframe.append({'Title':title,'Location':location,"Company":company,"Salary":salary,"Description":job_desc, "Occupation":occupation, "Sector":sector},ignore_index=True)

    dataframe.to_csv("media.csv",index=False) #edit

## Attemp to change proxy to avoid capcha 

In [ ]:
from lxml.html import fromstring
import requests
from itertools import cycle
import traceback

def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:10]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies


#If you are copy pasting proxy ips, put in the list below
#proxies = ['121.129.127.209:80', '124.41.215.238:45169', '185.93.3.123:8080', '194.182.64.67:3128', '106.0.38.174:8080', '163.172.175.210:3128', '13.92.196.150:8080']
proxies = get_proxies()
proxy_pool = cycle(proxies)

url = 'https://httpbin.org/ip'
for i in range(1,11):
    #Get a proxy from the pool
    proxy = next(proxy_pool)
    print("Request #%d"%i)
    try:
        response = requests.get(url,proxies={"http": proxy, "https": proxy})
        print(response.json())
    except:
        #Most free proxies will often get connection errors. You will have retry the entire request using another proxy to work. 
        #We will just skip retries as its beyond the scope of this tutorial and we are only downloading a single url 
        print("Skipping. Connnection error")